In [159]:
# For data
import numpy as np
from pandas import Series,DataFrame
import pandas as pd

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [160]:
train_filepath = './data/train.csv'
df = pd.read_csv(train_filepath)

In [161]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [162]:
df.describe()

/Users/riccardo/Documents/personal/kaggle-competitions/house-prices-prediction/env/lib/python3.6/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,NaN,7553.500000,5.000000,5.000000,1954.000000,1967.000000,NaN,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,NaN,9478.500000,6.000000,5.000000,1973.000000,1994.000000,NaN,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,NaN,11601.500000,7.000000,6.000000,2000.000000,2004.000000,NaN,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [163]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [164]:
house_prices = df.SalePrice

In [165]:
house_prices.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [166]:
df.count()

Id               1460
MSSubClass       1460
MSZoning         1460
LotFrontage      1201
LotArea          1460
Street           1460
Alley              91
LotShape         1460
LandContour      1460
Utilities        1460
LotConfig        1460
LandSlope        1460
Neighborhood     1460
Condition1       1460
Condition2       1460
BldgType         1460
HouseStyle       1460
OverallQual      1460
OverallCond      1460
YearBuilt        1460
YearRemodAdd     1460
RoofStyle        1460
RoofMatl         1460
Exterior1st      1460
Exterior2nd      1460
MasVnrType       1452
MasVnrArea       1452
ExterQual        1460
ExterCond        1460
Foundation       1460
                 ... 
BedroomAbvGr     1460
KitchenAbvGr     1460
KitchenQual      1460
TotRmsAbvGrd     1460
Functional       1460
Fireplaces       1460
FireplaceQu       770
GarageType       1379
GarageYrBlt      1379
GarageFinish     1379
GarageCars       1460
GarageArea       1460
GarageQual       1379
GarageCond       1379
PavedDrive

In [167]:
predictor_vars_x =  ['Id', 'LotArea', 'YrSold', 'MoSold']
predictor_vars = predictor_vars_x + ['SalePrice']

In [168]:
df_predictors = df[predictor_vars]

In [169]:
df_predictors.describe()

,Id,LotArea,YrSold,MoSold,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,10516.828082,2007.815753,6.321918,180921.195890
std,421.610009,9981.264932,1.328095,2.703626,79442.502883
min,1.000000,1300.000000,2006.000000,1.000000,34900.000000
25%,365.750000,7553.500000,2007.000000,5.000000,129975.000000
50%,730.500000,9478.500000,2008.000000,6.000000,163000.000000
75%,1095.250000,11601.500000,2009.000000,8.000000,214000.000000
max,1460.000000,215245.000000,2010.000000,12.000000,755000.000000


In [170]:
from sklearn.tree import DecisionTreeRegressor

In [171]:
prediction_model = DecisionTreeRegressor()

In [172]:
df_predictors_filtered = df_predictors.dropna(axis=0)

In [173]:
df_predictors_filtered.count()

Id           1460
LotArea      1460
YrSold       1460
MoSold       1460
SalePrice    1460
dtype: int64

In [174]:
prediction_model.fit(df_predictors_filtered[predictor_vars_x], df_predictors_filtered[['SalePrice']])

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [175]:
print(df_predictors_filtered.head())
print(prediction_model.predict(df_predictors_filtered[predictor_vars_x].head()))

   Id  LotArea  YrSold  MoSold  SalePrice
0   1     8450    2008       2     208500
1   2     9600    2007       5     181500
2   3    11250    2008       9     223500
3   4     9550    2006       2     140000
4   5    14260    2008      12     250000
[ 208500.  181500.  223500.  140000.  250000.]


In [176]:
from sklearn.model_selection import train_test_split

In [177]:
x_vars = df_predictors_filtered[predictor_vars_x]
y_vars = df_predictors_filtered[['SalePrice']]

In [178]:
train_x, val_x, train_y, val_y = train_test_split(x_vars, y_vars, random_state = 0)

In [179]:
prediction_model.fit(train_x, train_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [180]:
from sklearn.metrics import mean_absolute_error

In [181]:
predictions = prediction_model.predict(val_x)

In [182]:
mean_absolute_error(predictions, val_y)

72634.23835616438

In [183]:
def get_mae(max_leaf_nodes, predictors_train, predictors_val, targ_train, targ_val):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(predictors_train, targ_train)
    preds_val = model.predict(predictors_val)
    mae = mean_absolute_error(targ_val, preds_val)
    return(mae)

In [184]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_x, val_x, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  51214
Max leaf nodes: 50  		 Mean Absolute Error:  54153
Max leaf nodes: 500  		 Mean Absolute Error:  68835
Max leaf nodes: 5000  		 Mean Absolute Error:  69887


In [185]:
from sklearn.ensemble import RandomForestRegressor

In [186]:
forest_model = RandomForestRegressor(n_estimators=100)

In [187]:
forest_model.fit(train_x, train_y.values.ravel())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [188]:
predictions_train = forest_model.predict(val_x)

In [201]:
mean_absolute_error(predictions_train, val_y)

52694.864164383565

In [190]:
test_filepath = './data/test.csv'
df_test = pd.read_csv(test_filepath)
df_test_filtered = df_test[predictor_vars_x].dropna()

In [191]:
df_test_filtered.count()

Id         1459
LotArea    1459
YrSold     1459
MoSold     1459
dtype: int64

In [192]:
df_test_filtered.head()

,Id,LotArea,YrSold,MoSold
0,1461,11622,2010,6
1,1462,14267,2010,6
2,1463,13830,2010,3
3,1464,9978,2010,6
4,1465,5005,2010,1


In [193]:
predictions = forest_model.predict(df_test_filtered)

In [194]:
predictions.shape

(1459,)

In [195]:
df_test = df_test[['Id']]

In [196]:
df_test.shape

(1459, 1)

In [197]:
df_test['SalePrice'] = predictions

In [198]:
df_test.head()

,Id,SalePrice
0,1461,186864.82
1,1462,275073.34
2,1463,278735.73
3,1464,205067.32
4,1465,208202.28


In [199]:
df_test.to_csv('./data/results.csv', index=False)